In [22]:
from pathlib import Path
import cv2
from tqdm.notebook import tqdm
import numpy as np
from multiprocessing import Pool

In [23]:
dataset_path_str = '/data/smb_shared/@Dataset/HAdobe5K_resized1024'
foreground_folder_str = 'foreground_images'
mask_folder_str = 'masks'
foreground_cropped_folder_str = 'foreground_images_cropped'
mask_cropped_folder_str = 'masks_cropped'

In [25]:
dataset_path = Path(dataset_path_str)
assert dataset_path.is_dir()

foreground_folder_path = dataset_path / foreground_folder_str
assert foreground_folder_path.is_dir()
foreground_paths = sorted(foreground_folder_path.rglob('*.jpg'))

mask_folder_path = dataset_path / mask_folder_str
assert mask_folder_path.is_dir()
mask_paths = sorted(mask_folder_path.rglob('*.png'))

foreground_cropped_folder_path = dataset_path / foreground_cropped_folder_str
foreground_cropped_folder_path.mkdir(parents=True, exist_ok=False)
mask_cropped_folder_path = dataset_path / mask_cropped_folder_str
mask_cropped_folder_path.mkdir(parents=True, exist_ok=False)

In [26]:
def crop_image(mask_path):
    data_name = mask_path.stem.split('_')[0]

    mask = cv2.imread(str(mask_path))
    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # contours, hierarchy = cv2.findContours(
    #     binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # bounding_boxes = [cv2.boundingRect(cnt) for cnt in contours]

    illegal = True
    pic_h, pic_w = mask_gray.shape
    while illegal:
        illegal = False
        x = np.random.randint(pic_w)
        y = np.random.randint(pic_h)
        l = np.random.randint((pic_w + pic_h) / 6, (pic_w + pic_h) / 3)
        if x + l >= pic_w or y + l >= pic_h:
            illegal = True
            continue
        # for bbox in bounding_boxes:
        #     [bx, by, bw, bh] = bbox
        #     if x < bx+bw and x+l > bx and y < by+bh and y+l > by:
        #         illegal = True
        #         continue
    cropped_mask = 255 - mask_gray[y:y + l, x:x + l]
    cropped_mask_path = mask_cropped_folder_path / ('_'.join((data_name, '3')) + mask_path.suffix)
    cv2.imwrite(str(cropped_mask_path), cropped_mask)

    foreground_paths = foreground_folder_path.rglob(data_name + '*')
    for foreground_path in foreground_paths:
        foreground = cv2.imread(str(foreground_path))
        cropped_foreground = foreground[y:y + l, x:x + l]
        cropped_foreground_path = foreground_cropped_folder_path / (
                '_'.join((data_name, '3', foreground_path.stem.split('_')[2])) + foreground_path.suffix)
        cv2.imwrite(str(cropped_foreground_path), cropped_foreground)

In [ ]:
print('Mask num: ', len(mask_paths))
print('Foreground image num: ', len(foreground_paths))
with Pool() as pool:
    r = list(tqdm(pool.imap(crop_image, mask_paths), total = len(mask_paths)))